In [21]:
import pandas as pd
from csv import reader
import re
import os
from IPython.core.debugger import set_trace


In [22]:
HOMEDIR = os.getcwd()
DATA_DIR = 'data_sample'

filename = os.path.join(HOMEDIR, DATA_DIR, 'data.csv')

# View the data

In [23]:
with open(filename, 'r',encoding="latin1") as read_obj:
    csv_reader = reader(read_obj)
    for index, row in enumerate(csv_reader):    
        print(row)

['No. ', 'Molceular ', 'formula, name ', 'and pK value(s) ', 'ICC) ', 'Remarks ', 'Method ', 'Assessment ', 'Ref ']
['2004', 'CH40 Methanol ', '', '', '', '', '', '', '']
['', '15.5', '25', '', 'C3 ', 'Uncert. ', 'B8 ', '', '']
['', '15.09', '25', '', 'KIN ', 'Uncert. ', 'M126 ', '', '']
['2005', 'CH4Â°2 Methyl hydroperoxide ', '', '', '', '', '', '', '']
['', '11.5', '20', '', '05', 'Uncert. ', 'E27 ', '', '']
['2006', 'CH4S Methanethiol ', '', '', '', '', '', '', '']
['', '10.33', '25', '1% ethanol, gas solubility method ', '', 'Uncert. ', 'K57 ', '', '']
['2007', 'CHO-M Methane, trinitro- ', '', '', '', '', '', '', '']
['', '0.14', '20', 'In aqueous HC104 Ho scale ', '06', 'Uncert. ', 'T58a ', '', '']
['', '0.06', '25.5', 'Mixed constant ', '05', 'Uncert. ', 'H5 ', '', '']
['', '0.05', '9.6', '', '', '', '', '', '']
['', '0.23', '5', 'In aqueous HCl,Ho scale ', '06', 'Uncert. ', 'N39,S82 ', '', '']
['', '0.17', '20', '', '', '', '', '', '']
['', '0.11', '40', '', '', '', '', '', '']

# Class & function definitions

In [24]:
SPECIAL_TEMPS = '14.3','31.2','41.4','50.3','22.2','10.5','31.5','24.9','10.7','7-9','25.7','24.2','75.6','20.4', '30-36','10-12','18-20','31.5','25-30','20-22','26.5','0.35','not_stated','19.5','9.6','22.5','25.5','9.3','10.3','48.8', '15.3', '25.3','35.2','25.1','24.6','34.9','13.1','20.1','13.2','19.6','25.9','30.5', '12.1','18.8'
FOUR_DIGIT_PATTERN = r'(?<!.)(?<!\d)( )*\d{4}(?!\d)[abc]?'
UNCERT_TYPES = ('Uncert.','Uncert','Approx.','Approx','Rel','Rel.','V.uncert.','V.Uncert.')
METHOD_DICT = {'C1': 0,  'C2': 0, 'C3': 0,
             'E':0,'E1a': 0, 'E1b': 0, 'E1cg': 0, 'E1ch': 0, 'E1d': 0, 'E1e': 0,
             'E2a': 0, 'E2b': 0, 'E2c': 0, 
             'E3ag': 0, 'E3ah': 0, 'E3bg': 0, 'E3bh': 0, 'E3bq': 0, 'E3c': 0, 'E3d': 0, 'O5/E3bg':0, 'E3bg/O5' : 0,
             'O':0, 'O1': 0, 'O2': 0, 'O3': 0, 'O4': 0, 'O5': 0, 'O5a': 0, 'O6': 0, 'O7': 0, 
             'CAL': 0, 'CAT': 0, 'DIS': 0, 'FLU': 0, 'FLU/O5': 0, 'KIN': 0,
             'NMR': 0, 'RAM': 0, 'REF': 0, 'ROT': 0, 'SOL': 0,
             'R1a': 0, 'R1b': 0, 'R1a/b':0, 'R1c': 0, 'R1d': 0, 'R1e': 0,
             'R2a': 0, 'R2b': 0, 'R2c': 0,
             'R3a': 0, 'R3b': 0, 'R3c': 0, 'R3d': 0, 'R3f': 0, 'R3g': 0, 'R3h': 0,
             'R4': 0,
             'D1':0, 'D2':0, 'D3':0, 
             'K1a': 0, 'K1b':0, 'K1c': 0, 'K1d':0, 'K1e':0, 'K1f': 0,
             'K2a':0, 'K2b': 0, 'K2c':0, 'K2d':0, 'K2e':0}

class DataEntryBlock(object): # A block of text partially parsed from one entry # to another
    # Note to self: For numbers, account for repeats. There may be "repeats" blocks due to clerical errors
    def __init__(self, number = None, name  = None, content = [], discarded = None, entries = []):
        self.number = number
        self.name = name
        self.content = content # A list of entries.
        self.discarded = discarded
        self.entries = entries
        
    def __str__(self): 
        return "# {}: {} | {} \nEntries: {}\nDiscarded: {}\n".format(self.number, self.name, self.content, len(self.entries), self.discarded)
    
class DataEntry(object): # DataEntryBlocks "hold" multiple DataEntry objects.
    def __init__(self, pkas = [], T = [], remarks = None, method = [], assessment = [], ref = [], subtables= []):
        self.pkas = pkas
        self.T = T
        self.remarks = remarks
        self.method = method
        self.assessment = assessment
        self.ref = ref
        self.subtables = subtables

    def __str__(self): 
        return "pKa: {} | T: {} | Remarks: {} | Method: {} | Assessment: {} | Ref: {} \n".format(self.pkas, self.T, self.remarks, self.method, self.assessment, self.ref)


In [25]:
def is_mol_formula(word):
    elem_count = word.count('C') + word.count("\'") + word.count('H') + word.count('O') + word.count('F') + word.count('S') + word.count('N') + word.count('\"') + word.count('Â') + word.count('Ã') + word.count('°') + word.count('_') + 0.25*sum(c.isdigit() for c in word) + word.count('Br') + word.count('\"')
    elem_count = elem_count - word.count(',') - word.count('-') - word.count('(') - word.count(')') - word.count('=')
    if elem_count >= 2:
        return True
    else:
        return False

def find_and_replace(row):
    # A lazy, but very effective, way of getting typos and artifacts into an expected format.
    
    # expecting row = ['Entry here','B','C'...] where 'Entry here' is a w in row
    #    w = w.replace('=', ' =') for w in words if w.startswith('=')]
    for i, w in enumerate(row):
        row[i] = row[i].replace('I $', 'I =') 
        row[i] = row[i].replace('. .', '.') 
        row[i] = row[i].replace('-l-', '-1-') 
        row[i] = row[i].replace('I = 0. ', 'I = 0.') 
        row[i] = row[i].replace('I = 1. ', 'I = 1.') 
        row[i] = row[i].replace('I - 0', 'I = 0') 
        row[i] = row[i].replace('I 2.0', 'I=2.0') 
        row[i] = row[i].replace(' a ', ' = ')
        row[i] = row[i].replace('K1103', 'KNO3')
        row[i] = row[i].replace(' = ', '=') 
        row[i] = row[i].replace('Rl', 'R1') 
        row[i] = row[i].replace('/.Uncert', 'V.Uncert') 
        row[i] = row[i].replace('El', 'E1') 
        row[i] = row[i].replace('MaCl', 'NaCl') 
        row[i] = row[i].replace('KN03', 'KNO3')
        row[i] = row[i].replace('KM03', 'KNO3')
        row[i] = row[i].replace('aCIOs', 'aClO4') 
        row[i] = row[i].replace(' 1 (','1(') 
        row[i] = row[i].replace(' 1(Na','1(Na') 
        row[i] = row[i].replace(' 1(K','1(K') 
        row[i] = row[i].replace('KC1','KCl') 
        row[i] = row[i].replace('Kla','K1a') 
        row[i] = row[i].replace('KHO3','KNO3')
        row[i] = row[i].replace('V. Uncert','V. Uncert') 
        row[i] = row[i].replace('NaC1','NaCl') 
        row[i] = row[i].replace('Nactog','NaClO4') 
        row[i] = row[i].replace('flactog','NaClO4') 
        row[i] = row[i].replace('Blll','B111')  
        row[i] = row[i].replace('Mlll','M111')  
        row[i] = row[i].replace('Gll','G11')  
        row[i] = row[i].replace('ybbcox','')  
        row[i] = row[i].replace('ybblox','')  
        row[i] = row[i].replace('yoblox','')  
        row[i] = row[i].replace('ybbLox','')  
        row[i] = row[i].replace('C10','ClO') 
        row[i] = row[i].replace('Cl0','ClO') 
        row[i] = row[i].replace('E3bg/05','E3bg/O5') 
        row[i] = row[i].replace('N02','NO2') 
        row[i] = row[i].replace('pky', 'pk1') 
        row[i] = row[i].replace(', p', ' p') 
        row[i] = row[i].replace('07,R', 'O7,R') 
        row[i] = row[i].replace('05,R', 'O5,R') 
        row[i] = row[i].replace(' (trans)', '(trans)') 
        row[i] = row[i].replace(' (cis)', '(cis)') 
        row[i] = row[i].replace('(C0', '(CO') 
        row[i] = row[i].replace('(Contd)', '') 
        row[i] = row[i].replace('(contd)', '') 
        row[i] = row[i].replace('pKg', 'pk1') 
        row[i] = row[i].replace('pk,', 'pk1') 
        row[i] = row[i].replace('Mll,', 'M11') 
        row[i] = row[i].replace('Gll', 'G11') 
        row[i] = row[i].replace(' <', '<') 
        row[i] = row[i].replace(' >', '>') 
        row[i] = row[i].replace('05a', 'O5a') 
        row[i] = row[i].replace('thy1', 'thyl') 
        row[i] = row[i].replace('oly1', 'olyl') 
        row[i] = row[i].replace('E3pa', '') 
        row[i] = row[i].replace('E2pa', '') 
        row[i] = row[i].replace('E3po', '') 
        row[i] = row[i].replace('opy1', 'opyl') 
        row[i] = row[i].replace('eny1', 'enyl') 
        row[i] = row[i].replace('exy1', 'exyl') 
        row[i] = row[i].replace('ty1', 'tyl') 
        row[i] = row[i].replace('ly1', 'lyl') 
        row[i] = row[i].replace('ry1', 'ryl') 
        row[i] = row[i].replace('ox0', 'oxo') 
        row[i] = row[i].replace('0x', 'Ox') 
        row[i] = row[i].replace('c 0', 'c=0') 
        row[i] = row[i].replace('C 0', 'C=0') 
        row[i] = row[i].replace('I 0', 'I=0') 
        row[i] = row[i].replace('I  0', 'I=0') 
        row[i] = row[i].replace('c  0', 'c=0') 
        row[i] = row[i].replace('C  0', 'c=0') 
        row[i] = row[i].replace('c104', 'ClO4') 
        row[i] = row[i].replace('ciog', 'ClO4') 
        row[i] = row[i].replace('1laC', 'NaC') 
        row[i] = row[i].replace('cio4', 'ClO4') 
        row[i] = row[i].replace('Nacion', 'NaClO4') 
        row[i] = row[i].replace('i!', 'N') 
        row[i] = row[i].replace('cioa', 'ClO4')
        row[i] = row[i].replace('(maclo', '(NaClO')
        row[i] = row[i].replace('ilacioq', 'NaClO4')
        row[i] = row[i].replace('ilaC1', 'NaCl')
#        row[i] = row[i].replace('0-', 'O-') 
        row[i] = row[i].replace('capa', '') 
        row[i] = row[i].replace('pK of electron', 'p\'k of electron') 
        if row[i] in ['01','02','03','04','05','06','07','08','09', '010', '011', '012', '013', '014', '015', '016','017','018','019','020','021']:
            row[i] = w.replace('0','O')
        if row[i] in ['01 ','02 ','03 ','04 ','05 ','06 ','07 ','08 ','09 ', '010 ', '011 ', '012 ', '013 ', '014', '015 ', '016 ','017 ','018 ','019 ','020 ','021 ']:
            row[i] = w.replace('0','O')
#        if w in ['1','2','3','4','6','7','8','9']:
#            words[count] = "O" + str(w)
        if row[i] == '0':
            row[i] = "O"
        if row[i] == 'Tla':
            row[i] = 'T1a'
        if row[i] == '115':
            row[i] = 'I15'
        if row[i] == '120':
            row[i] = 'I20'
        row[i] = row[i].replace('(H20)','(H2O)')
        row[i] = row[i].replace('(D20)','(D2O)')
        row[i] = row[i].replace(' (H2O)','(D2O)')
        row[i] = row[i].replace(' (D2O)','(D2O)')
        row[i] = row[i].replace(' (co)','(co)')
        row[i] = row[i].replace(' (cooh)','(cooh)')
        row[i] = row[i].replace(' (COOH)','(COOH)')
        row[i] = row[i].replace(' (NH)','(NH)')
        row[i] = row[i].replace(' (OH)','(OH)')
        row[i] = row[i].replace('kno3', 'KNO3')
        row[i] = row[i].replace('onditions not stated', 'onditions-not-stated') 
        row[i] = row[i].replace('not stated', 'not_stated')
        row[i] = row[i].replace('Not stated', 'not_stated')
    return row

In [26]:
def load_data():
    rows = []
    with open(filename, 'r',encoding="latin1") as read_obj:
        csv_reader = reader(read_obj)
        for _, row in enumerate(csv_reader):
            rows.append(row)
    return rows
        
def preprocess_data(data):    
# A few functions:
# (1) Removes useless header rows.
# (2) Splits entry #s like '2002Propanoic' into '2002', 'Propanoic' separate cells
# (3) Checks for - and removes - molecular formulas, which are often improperly read into the sheet
# (4) Removes completely empty rows.
# This is all to make the data easier to parse into DataEntryBlock objects.
    list_of_lists = []
    for indexline, row in enumerate(data):
        #Looping through rows
        
        line_type = None

        for index, item in enumerate(row):
            # Find-and-replace subroutine
            row = find_and_replace(row)   

            # Looping through words
            fragment = None
            row[index] = item.strip()
            if index == 0:
                match = re.search(FOUR_DIGIT_PATTERN,item)
                if match != None:
                    # Successful match.                     
                    # See if there are any nonmatches e.g. 'Ahh2012test' --> ['Ahh','test']
                    row[0] = ''
                    nonmatches = re.split(FOUR_DIGIT_PATTERN, item)
                    nonmatches = [i for i in nonmatches if not (i in ['',' '])] # if no nonmatches, just get rid of it

                    if len(nonmatches) > 0:
                        nonmatches.reverse()
                        try:
                            row.insert(0, ''.join(nonmatches))
                        except TypeError:
                            print(nonmatches)
                    row.insert(0,''.join(match.group()))
                    line_type = 'first'

            # Check for the mol formula here
        for index, item in enumerate(row):
            if line_type == 'first':
                # Try to split up molecular formula from rest of string. Delete it if possible.
                split_string = item.split() # Ex: 'C2HO2F3 Acetic acid, trifluoro-' -> ['C2H02F3', 'Acetic', 'acid,', 'trifluoro-']

                if len(split_string) == 1: # this must go first for case of word like "CH2 Acetic acid"
                    if is_mol_formula(split_string[0]):
                        row.remove(row[index])
                
                if len(split_string) > 1:
                    for word in split_string:
                        if is_mol_formula(word):
#                            molec_formula = word
                            split_string.remove(word)
                            fragment = ' '.join(split_string)
                            row[index] = fragment
                            break                

               
                    
        if any(e in row for e in ['Remarks','No.','Molecular','formula','Method','Assessment', 'Ref','Ref.']) or all(e == '' for e in row): 
            print("Dropping linerow {}".format(indexline))
        else:
            list_of_lists.append(row)

    return list_of_lists

def process_data(data):
    # Turns a csv into a list of block datas. Assumes that entry #s are always in the first column.
    block = DataEntryBlock(content = [], entries=[], name = None)
    block_list = []
    for _, row in enumerate(data):
        for index, item in enumerate(row):
            indices_to_del = []
            if index == 0:
            # Try to find a number of format "XXXX" in the first column.
            # If successful, split on the string if possible, and create a new block
            # Otherwise try to add it to the block.
                match = re.search(FOUR_DIGIT_PATTERN,item)
                if match != None:
                    block_list.append(block) # Add previous block because it's done parsing
                    block = DataEntryBlock(number = match.group(), content = [], entries=[])
                    indices_to_del.append(0)

            # THE LAST THING TO BE EXECUTED: Delete all the indices we wanted to.
            if len(indices_to_del) > 0:
                for indice in indices_to_del:
                    del row[indice]
        
        block.content.append(row)

    block_list.append(block) # Add last block
    block_list.remove(block_list[0])# Remove first block
    return block_list

# Run the functions to process data into block form

In [27]:
# idea: As iterating through list, try to extract an entry # from first 3 columns
# Also create a DataEntry class and flag when there is a missing entry.
 
data = load_data()
alldata = preprocess_data(data)
#alldata = preprocess_data(alldata) # Process twice 


Dropping linerow 0
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]


In [28]:
for row in alldata:
    print(row)

['2004', '', 'Methanol', '', '', '', '', '', '', '']
['', '15.5', '25', '', 'C3', 'Uncert.', 'B8', '', '']
['', '15.09', '25', '', 'KIN', 'Uncert.', 'M126', '', '']
['2005', '', 'Methyl hydroperoxide', '', '', '', '', '', '', '']
['', '11.5', '20', '', 'O5', 'Uncert.', 'E27', '', '']
['2006', '', 'Methanethiol', '', '', '', '', '', '', '']
['', '10.33', '25', '1% ethanol, gas solubility method', '', 'Uncert.', 'K57', '', '']
['2007', '', 'Methane, trinitro-', '', '', '', '', '', '', '']
['', '0.14', '20', 'In aqueous HClO4 Ho scale', 'O6', 'Uncert.', 'T58a', '', '']
['', '0.06', '25.5', 'Mixed constant', 'O5', 'Uncert.', 'H5', '', '']
['', '0.05', '9.6', '', '', '', '', '', '']
['', '0.23', '5', 'In aqueous HCl,Ho scale', 'O6', 'Uncert.', 'N39,S82', '', '']
['', '0.17', '20', '', '', '', '', '', '']
['', '0.11', '40', '', '', '', '', '', '']
['', '0.02', '60', 'Thermodynamic quantities are derived from the results', '', '', '', '', '']
['2008', '', 'Methane, dinitro-', '3.63', '20', ''

In [29]:
blockdata = process_data(alldata)

In [30]:
for block in blockdata:
    print(block)

# 2004: None | [['', 'Methanol', '', '', '', '', '', '', ''], ['', '15.5', '25', '', 'C3', 'Uncert.', 'B8', '', ''], ['', '15.09', '25', '', 'KIN', 'Uncert.', 'M126', '', '']] 
Entries: 0
Discarded: None

# 2005: None | [['', 'Methyl hydroperoxide', '', '', '', '', '', '', ''], ['', '11.5', '20', '', 'O5', 'Uncert.', 'E27', '', '']] 
Entries: 0
Discarded: None

# 2006: None | [['', 'Methanethiol', '', '', '', '', '', '', ''], ['', '10.33', '25', '1% ethanol, gas solubility method', '', 'Uncert.', 'K57', '', '']] 
Entries: 0
Discarded: None

# 2007: None | [['', 'Methane, trinitro-', '', '', '', '', '', '', ''], ['', '0.14', '20', 'In aqueous HClO4 Ho scale', 'O6', 'Uncert.', 'T58a', '', ''], ['', '0.06', '25.5', 'Mixed constant', 'O5', 'Uncert.', 'H5', '', ''], ['', '0.05', '9.6', '', '', '', '', '', ''], ['', '0.23', '5', 'In aqueous HCl,Ho scale', 'O6', 'Uncert.', 'N39,S82', '', ''], ['', '0.17', '20', '', '', '', '', '', ''], ['', '0.11', '40', '', '', '', '', '', ''], ['', '0.02', 

# Process from block form

In [31]:
def is_pka (value):
    pattern = re.compile("(\d){1,3}[.](\d){1,4}[ ]?(\([^\)]*\)){1}")
    try:
        if re.search(pattern, value) and 'I' not in value and 'k' not in value and 'K' not in value and 'Na' not in value and '?' not in value and 'L' not in value:
            return True
        float(value)
        if '.' in value and abs(float(value)) > 0.04 and not is_special_temp(value) and float(value) < 25.0: 
            return True
        else:
            return False
    except ValueError:
        return False

def is_temp(value):
    try:
        int(value)
        if int(value) < 500:
            return True
        else:
            return False
    except:
        if is_special_temp(value):
            return True
        try:
            z = value.split('+')
            for a in z:
                int(b)
            return True
        except:
            return False
    
    return False

def is_special_temp(value):
    pattern = re.compile("(\d){1,3}[+](\d){1,2}")
    pattern_q_mark = re.compile("(\d){1,3}(\([?]\))")
    if (value in SPECIAL_TEMPS) or re.search(pattern, value) or re.search(pattern_q_mark, value):
        return True
    else:
        return False

In [32]:
def populate_entries(blockdata):
    for block in blockdata:
        # Looping through blocks    

        for linenumber, line in enumerate(block.content):
            entry = DataEntry(pkas = [], T = [], remarks = None, method = [], ref = [], assessment = [], subtables = [])
            method_list = []
            method_indices = []
            ref_list = []
            ref_indices = []
            pkas = []
            pkas_indices = []
            pkas_types = []
            pkas_types_indices = []
            assessment_list = []
            T_list = []
            T_indices = []
            pkas_dict = {}
            # Looping through lines in a single block
            
            # FIRST LOOP: Try to identify very specific edge cases.
            # Edge case 1: "#,#-____" e.g. 2,2-dimethylphenyl e.g.
            pattern = re.compile("(\d)?(\()?((\d\,)+)?([A-z]|\d)-[A-z\d\(\)[\]\-,]+") # e.g. matches 1,1-dimethyl; 2,2-mercapto-4-silica; 3,3,3-methyl-4-silica
            for index, content in enumerate(line):
                split_iupac_words = content.split()

                for ct,word in enumerate(split_iupac_words):
                    try:
                        matching = pattern.match(word).group()
                        if block.name == None:
                            block.name = matching
                            split_iupac_words[ct] = split_iupac_words[ct].replace(matching, '', 1)
                        elif block.name.contains(matching):
                            pass
                        else:
                            block.name = block.name + matching
                    except AttributeError:
                        pass

                block.content[linenumber][index] = ' '.join(split_iupac_words)   

                # Edge case 2: 'pk = ' or 'pK = ' in string
                pk_edge_cases = ['pK = ', 'pk = ']
                for case in pk_edge_cases:
                    block.content[linenumber][index] = block.content[linenumber][index].replace(case, 'pk=')


            # SECOND LOOP: Get "easy ones" a.k.a. uncert. types, methods, ref
            pattern = re.compile("^[A-NP-Z]{1}[0-9]{1,3}[a-z]{0,1}$")
            for index, content in enumerate(line):

                # CHECK 1: If word is in methods or is a temperature.
                # Split word by commas only for temperatures
                if (index <= 2) and linenumber == 0:
                    split_string = ' '
                else:
                    split_string = '[, ]'
                remaining_words = re.split(split_string,line[index])
                for ct,word in enumerate(remaining_words):
                    if remaining_words[ct] in list(METHOD_DICT.keys()):
                        method_list.append(word)
                        method_indices.append([index,ct])
                        remaining_words[ct] = remaining_words[ct].replace(word, '')
                    if is_temp(remaining_words[ct]):                        
                        T_list.append(word)
                        T_indices.append([index,ct])
                        remaining_words[ct] = remaining_words[ct].replace(word, '')
                    if remaining_words[ct] in UNCERT_TYPES: # search for uncertainty types
                        assessment_list.append(word)
                        remaining_words[ct] = remaining_words[ct].replace(word, '')
                    try: # search for refs
                        matching = pattern.match(remaining_words[ct]).group()
                        if matching not in ['I','No','C','N','D20','H20'] and matching not in list(METHOD_DICT.keys()) and ' '.join(line).find("Other values") == -1:
                            remaining_words[ct] = remaining_words[ct].replace(matching, '')
                            ref_list.append(matching)
                            ref_indices.append([index, ct])
                    except AttributeError:
                        pass
                    
                line[index] = ' '.join(remaining_words)            

            
            # Edge case: '05' improperly parsed as '5' and moved to T, and there are no methods, but there are other tags for the line.
            # OR 
            if len(T_list) < 3 and len(method_list) == 0 and (entry.assessment != None or entry.ref != None):
                numerals_to_check = ['01','02','03','04','05','06','07','08','09']
                nums_to_check = ['1','2','3','4','5','6','7','8','9']
                
                for numeral in nums_to_check:
                    if numeral in T_list: # Can make this a function if this is a general problem
                        method_list.append('O' + numeral)
                        ind = T_list.index(numeral)
                        T_list.remove(numeral)
                        T_indices.remove(T_indices[ind])

                for numeral in numerals_to_check:
                    if numeral in T_list: # Can make this a function if this is a general problem
                        fixed_str = numeral.replace('0','O')
                        method_list.append(fixed_str)
                        ind = T_list.index(numeral)
                        T_list.remove(numeral)
                        T_indices.remove(T_indices[ind])

                        
            # Update list
            entry.method = method_list
            entry.method_indices = method_indices
            entry.T = T_list
            entry.T_indices = T_indices
            entry.ref = ref_list
            entry.ref_indices = ref_indices
            entry.assessment = assessment_list

            # update the 'max iter' for temp index
            try:
                maxiter = T_indices[0][0] + T_indices[0][1]
            except IndexError:
                maxiter = 4

            # THIRD LOOP: Get pKas. Only loop up to the temperatures index; failing this, to the fourth column, UNLESS there are many pKas, in which case this is probably a table.
            for index, content in enumerate(line):
                # CHECK 1: If word is in pkas.
                # Split word by spaces and then commas
                split_on = [' ', ',']
                split_text = line[index]
                for split in split_on:
                    remaining_words = re.split(split,split_text)
                    for ct,word in enumerate(remaining_words):
                        if is_pka(word):
                            pkas.append(word)
                            pkas_indices.append([index,ct])
                            remaining_words[ct] = remaining_words[ct].replace(word, '',1)
                            if len(pkas) > 2: # If we're on a roll, this is probably a table
                                maxiter = maxiter + 1 
                        elif ('pk' in word or 'pK' in word or 'Pk' in word or 'PK' in word) and ('pk=' not in word and '(pK' not in word and '(pk' not in word): 
                            if word in pkas_types:
                                pkas_types.append(word + '_' + str(index) + '_' + str(ct))
                            else:
                                pkas_types.append(word)
                            pkas_types_indices.append([index,ct])
                            remaining_words[ct] = remaining_words[ct].replace(word, '',1)
                        elif split == ',' and ct > 0:
                            remaining_words[ct] = ',' + remaining_words[ct]
                        elif split == ' ' and ct > 0:
                            remaining_words[ct] = ' ' + remaining_words[ct]
                            
                    split_text = ''.join(remaining_words)
                            

                # Clear remaining words
                line[index] = ' '.join(remaining_words)
                
                if index == maxiter:
                    break
                
            # Update pkas
            for count,val in enumerate(pkas):
                try:
                    pkas_dict[pkas_types[count]] = val 
                except IndexError:
                    pkas_dict['__'+str(count)] = val
            
            entry.pkas = pkas_dict            
                
            # Penultimately, try to get name from first line
            if linenumber == 0:
                if block.name == None: 
                    block.name = ''
                block.name = block.name + ' '.join(line[0:3]) # do this addition in case of edge case detection earlier in loop
                line[0] = ''
                line[1] = ''
                line[2] = ''
                block.name = block.name.strip()
                block.name = block.name.lstrip('-')
            
            # Save the rest as remarks.
            entry.remarks = ' '.join(line).strip()
            
            if any([len(entry.pkas) > 0, len(entry.T) > 0, len(entry.remarks) > 0, len(entry.method) > 0, entry.assessment != None, entry.ref != None ] ):
                block.entries.append(entry)
            
            print(block.name)
            print(line)
            print(entry)

    return blockdata

blockdata = populate_entries(blockdata)

Methanol
['', '', '', '', '', '', '', '', '']
pKa: {} | T: [] | Remarks:  | Method: [] | Assessment: [] | Ref: [] 

Methanol
['', '', '', '', '', '', '', '', '']
pKa: {'__0': '15.5'} | T: ['25'] | Remarks:  | Method: ['C3'] | Assessment: ['Uncert.'] | Ref: ['B8'] 

Methanol
['', '', '', '', '', '', '', '', '']
pKa: {'__0': '15.09'} | T: ['25'] | Remarks:  | Method: ['KIN'] | Assessment: ['Uncert.'] | Ref: ['M126'] 

Methyl hydroperoxide
['', '', '', '', '', '', '', '', '']
pKa: {} | T: [] | Remarks:  | Method: [] | Assessment: [] | Ref: [] 

Methyl hydroperoxide
['', '', '', '', '', '', '', '', '']
pKa: {'__0': '11.5'} | T: ['20'] | Remarks:  | Method: ['O5'] | Assessment: ['Uncert.'] | Ref: ['E27'] 

Methanethiol
['', '', '', '', '', '', '', '', '']
pKa: {} | T: [] | Remarks:  | Method: [] | Assessment: [] | Ref: [] 

Methanethiol
['', '', '', '1% ethanol  gas solubility method', '', '', '', '', '']
pKa: {'__0': '10.33'} | T: ['25'] | Remarks: 1% ethanol  gas solubility method | Metho

In [33]:
def show_blockdata(blockdata):
    for block in blockdata:
        print(block)
        for entry in block.entries:
            print(entry)
        print('\n')

show_blockdata(blockdata)

# 2004: Methanol | [['', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '']] 
Entries: 3
Discarded: None

pKa: {} | T: [] | Remarks:  | Method: [] | Assessment: [] | Ref: [] 

pKa: {'__0': '15.5'} | T: ['25'] | Remarks:  | Method: ['C3'] | Assessment: ['Uncert.'] | Ref: ['B8'] 

pKa: {'__0': '15.09'} | T: ['25'] | Remarks:  | Method: ['KIN'] | Assessment: ['Uncert.'] | Ref: ['M126'] 



# 2005: Methyl hydroperoxide | [['', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '']] 
Entries: 2
Discarded: None

pKa: {} | T: [] | Remarks:  | Method: [] | Assessment: [] | Ref: [] 

pKa: {'__0': '11.5'} | T: ['20'] | Remarks:  | Method: ['O5'] | Assessment: ['Uncert.'] | Ref: ['E27'] 



# 2006: Methanethiol | [['', '', '', '', '', '', '', '', ''], ['', '', '', '1% ethanol  gas solubility method', '', '', '', '', '']] 
Entries: 2
Discarded: None

pKa: {} | T: [] | Remarks:  | Method: [] | Assessment: [] | Ref: [] 

pKa: {'

In [34]:
# Loop and get rid of blank rows
# Sort of a post-processing step
def clean_empty_rows(blockdata):
    for blocknumber, block in enumerate(blockdata):

        # Also clear spaces in the word
        blockdata[blocknumber].name = re.sub(' +', ' ', block.name)
        blockdata[blocknumber].name = blockdata[blocknumber].name.rstrip(',')
        blockdata[blocknumber].name = blockdata[blocknumber].name.rstrip(' ')
        blockdata[blocknumber].name = blockdata[blocknumber].name.rstrip(',')
        blockdata[blocknumber].name = re.sub(' +', ' ', block.name)
        blockdata[blocknumber].name = blockdata[blocknumber].name.replace(' ,', ',')
        blockdata[blocknumber].name = blockdata[blocknumber].name.replace('- -', '-')
        blockdata[blocknumber].name = blockdata[blocknumber].name.replace(' - ', '-')
        blockdata[blocknumber].name = blockdata[blocknumber].name.replace(' -', '-')
        blockdata[blocknumber].name = blockdata[blocknumber].name.replace(' . ', '.')

        for count, entry in enumerate(block.entries):
            combined_data = [entry.pkas, entry.T, [entry.remarks], entry.method, entry.assessment, entry.ref]
            unique_values = list(set([item for sublist in combined_data for item in sublist]))

            # (1) Eliminate empty rows
            if len(unique_values) == 1 and unique_values[0] == '':
                blockdata[blocknumber].entries.remove(entry)
            
            # need regex expression matching ', [d]'

        for count, entry in enumerate(block.entries): # separate loop in case stuff got deleted
            # (2) Clean up a bit
            blockdata[blocknumber].entries[count].remarks = blockdata[blocknumber].entries[count].remarks.replace(' . ', '.')
            blockdata[blocknumber].entries[count].remarks = blockdata[blocknumber].entries[count].remarks.replace(' .', '.')

    return(blockdata)

blockdata = clean_empty_rows(blockdata)

In [35]:
def decompose_pkas(pka_dict):
    pka_types = list(pka_dict.keys())
    pkas = list(pka_dict.values())
    return pka_types, pkas


## Post-post processing: Operating 'in the cleaned entries'

In [36]:
# Subtables and post-processing of remarks
EDGE_CASE_REMARKS = ['I -', 'I =', 'I=','C=','C =', 'C  ', 'I ', 'm = '
                     'Concentration', 'Calculated','calculated', 'conditions',
                     'Conditions', 'aq.', 'Room temp', 'Mixed ', 'mixed ', 'not stated',
                     'Compound', 'No detail', "not stated", "m=","I=",'c=', 'C=', 'c =', 'Values', 'other',
                     'Probably', 'C<', 'C <', 'I<', 'I <', ' not ', ' in 0', 'room temp']

def postpost_process(blockdata):
    for block in blockdata:
        if len(block.entries) == 0:
            print("Skipping")
            continue
        assume_subtable = False
        subtable_type = None

        # (1) Identify subtables
        for count, entry in enumerate(block.entries):
            combined_data = [entry.pkas, entry.T, [entry.remarks], entry.method, entry.assessment, entry.ref]
            unique_values = list(set([item for sublist in combined_data for item in sublist]))

            if (("Variation " in entry.remarks and "with" in entry.remarks) or "with pressure" in entry.remarks):
                if "with pressure" in entry.remarks:
                    subtable_type = "pressure"
                assume_subtable = True
                if count > 0:
                    subtable_index = count-1
                else:
                    subtable_index = count
                block.entries[subtable_index].subtables.append(entry.remarks)
                block.entries[count].remarks = ''
                first_loop = True

            if assume_subtable == True:
                added_pressure = False
#                if sum([int(entry.assessment == []), int(entry.method == []), int(entry.ref == [])]) >= 2:
                if len(entry.pkas) > 2: # Long list of pkas 
                    block.entries[subtable_index].subtables.append(entry.pkas)
                    block.entries[count].pkas = {}
                if len(entry.T) > 2: # Long list of temps
                    block.entries[subtable_index].subtables.append(entry.T)
                    block.entries[count].T = []

                numbers = sum(c.isdigit() for c in entry.remarks)
                letters = sum(c.isalpha() for c in entry.remarks)
                if ("(atm)" in entry.remarks or "(bar)" in entry.remarks) or (subtable_type == 'pressure' and numbers > 3 and '=' not in entry.remarks and numbers > letters): # Or if the text includes some indication of pressure
                    block.entries[subtable_index].subtables.append(entry.remarks)
                    block.entries[count].remarks = ''
                    added_pressure = True
                if len(entry.T) <= 2 and len(entry.pkas) <= 2 and added_pressure == False and first_loop == False and len(unique_values) <= 1:
                    assume_subtable = False    
                    subtable_type = None
                first_loop = False

        # (2) Move certain edge cases from name to remarks if detected
        case_list = []
        for case in EDGE_CASE_REMARKS: 
            ind_find = block.name.find(case)
            if ind_find != -1:
                case_list.append(ind_find)
        if len(case_list) > 0:
            name_str = block.name[0:min(case_list)]
            remarks_str = block.name[min(case_list):]
            block.name = name_str
            if block.entries[0].remarks == '':
                block.entries[0].remarks = remarks_str
            else:
                block.entries[0].remarks = block.entries[0].remarks + '\n' + remarks_str

        # (3) Edge case if '=' in remarks and pkas > pk
        for entry in block.entries:
            matching_indices = [m.start() for m in re.finditer('=', entry.remarks)]
            if len(matching_indices) > 0:
                pka_types, pkas = decompose_pkas(entry.pkas)
                for i, pka_type in reversed(list(enumerate(pka_types))):
                    if '__' in pka_type and len(pka_types) < len(pkas):
                        pka_types.remove(pka_type)
                        del pkas[i]
                        entry.pkas.pop(pka_type)
                    for j, pka in reversed(list(enumerate(pkas[len(pkas):]))):
                        res = list(entry.remarks)
                        try:
                            res.insert(matching_indices[j], pka)
                        except IndexError:
                            res.insert(matching_indices[len(matching_indices)-1], pka)
                        res = ''.join(res)

        # (4) Handle multiple temperatures
        for entry in block.entries:
            while len(entry.T) > 1:

                # Case 0: if there's "not_stated", then that's it
                for temp in entry.T:
                    if temp == 'not_stated':
                        entry.T.remove('not_stated')
                        entry.remarks = entry.remarks + ' '.join(entry.T)
                        entry.T = ['not_stated']
                        break
                
                try:
                    z = [int(float(k)%3) for k in entry.T]
                except: # wow this is really bad coding practice...
                    z = []

                # Case 1: pKa not present
                if len(entry.pkas) == 0 and len(entry.T) > 1:
                    # check if there is a 'special' T. If so, move this to the pKa
                    for temp in entry.T:
                        if is_special_temp(temp):
                            entry.pkas['__0c'] = temp
                            entry.T.remove(temp)

                # Case 2: Method not present
                if len(entry.method) == 0 and len(entry.T) > 1:
                    for temp in entry.T:
                        try:
                            if int(temp) > 0 and int(temp) < 10:
                                entry.method.append('O' + str(int(temp)))
                                entry.T.remove(temp)
                                break
                        except ValueError:
                            pass
                
                # Case 3: negative temp is a remarks thing
                if ('-1' in entry.T or '-2' in entry.T) and len(entry.T) > 1:
                    for temp in entry.T:
                        if temp == "-1" or temp == "-2":
                            entry.remarks = entry.remarks + "temp"
                            entry.T.remove(temp)
                            break

                # Case 4: there's a '25' in the 
                if '25' in entry.T and len(entry.T) > 1:
                    entry.T.remove('25')
                    entry.remarks = entry.remarks + ' '.join(entry.T)
                    entry.T = ['25']

                # Case 5: just take the lowest %5 one
                if 0 in z and len(entry.T) > 1:
                    epsilon = 0.001
                    for temp in entry.T:
                        if float(temp) % 5 < epsilon:
                            entry.T.remove(temp)
                            entry.remarks = entry.remarks + ' '.join(entry.T)
                            entry.T = [temp]
                            break

                # Case 6: just take the 1st temperature found
                if len(entry.T) > 1:
                    entry.remarks = entry.remarks + ' '.join(entry.T[1:])
                    entry.T = [entry.T[0]]




        # (5) Add remarks of near-empty rows to previous rows
        for count, entry in reversed(list(enumerate(block.entries))):
            if sum([len(entry.pkas), len(entry.T), len(entry.method), len(entry.assessment), len(entry.ref)]) == 0 and entry.remarks != '':
                if count > 0:
                    if block.entries[count-1].remarks == '':
                        block.entries[count-1].remarks = entry.remarks
                    else:
                        block.entries[count-1].remarks = block.entries[count-1].remarks + '\n' + entry.remarks
                elif count == 0:
                    block.name = block.name + ' ' + entry.remarks 
                
                block.entries[count].remarks = ''
                
        # (6) Another edge case. What if there's no pKa, but 1 special temperature?
        # Move it from temp to pKa
        for entry in block.entries:
            if len(entry.pkas) == 0 and len(entry.T) == 1:
                if is_special_temp(entry.T[0]):
                    entry.pkas['__0'] = entry.T[0]
                    entry.T.remove(entry.T[0])
                    

    return blockdata

blockdata = postpost_process(blockdata)
blockdata = clean_empty_rows(blockdata)
blockdata = postpost_process(blockdata)

In [37]:
blockdata = clean_empty_rows(blockdata)
blockdata = clean_empty_rows(blockdata)

# Create a new CSV

In [38]:
list_of_dicts = []

df = pd.DataFrame(columns=['Entry #', 'IUPAC', 'pKa type','pKa','T','Remarks','Method','Assessment','Ref', '(Subtable)'])

num = 0
# we would prefer to do this dict-by-dict, but this is OK in a pinch
for block in blockdata:
    l = [None]*10
    l[0] = block.number
    l[1] = block.name
    df.loc[num] = l
    for entry in block.entries:
        l[0] = None
        l[1] = None
        l[2] = ', '.join(list(entry.pkas.keys()))
        l[3] = ', '.join(list(entry.pkas.values()))
        l[4] = ', '.join(entry.T)
        l[5] = entry.remarks
        l[6] = ', '.join(entry.method)
        l[7] = ', '.join(entry.assessment)
        try:
            l[8] = ', '.join(entry.ref)
        except:
            l[8] = ''
        if entry.subtables != []:
            l[9] = entry.subtables
        else:
            l[9] = ''
        num = num + 1
        df.loc[num] = l
    num = num + 1

In [39]:
df.head(999)

,Entry #,IUPAC,pKa type,pKa,T,Remarks,Method,Assessment,Ref,(Subtable)
0,2004,Methanol,None,None,None,None,None,None,None,None
1,None,None,__0,15.5,25,,C3,Uncert.,B8,
2,None,None,__0,15.09,25,,KIN,Uncert.,M126,
3,2005,Methyl hydroperoxide,None,None,None,None,None,None,None,None
4,None,None,__0,11.5,20,,O5,Uncert.,E27,
5,2006,Methanethiol,None,None,None,None,None,None,None,None
6,None,None,__0,10.33,25,1% ethanol gas solubility method,,Uncert.,K57,
7,2007,"Methane, trinitro-",None,None,None,None,None,None,None,None
8,None,None,__0,0.14,20,In aqueous HClO4 Ho scale,O6,Uncert.,T58a,
9,None,None,__0,0.06,25.5,Mixed constant,O5,Uncert.,H5,


In [40]:
filename_out = "test_sample_out.csv"
df.to_csv(filename_out,index=False)